In [2]:
import pygame 


In [6]:
# Define some colors
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
GREEN = (0, 255, 0)
RED = (255, 0, 0)
pygame.init()
# Set the width and height of the screen [width, height] 
# (0,0) is upper-right corner
size = (700, 500)
screen = pygame.display.set_mode(size)
pygame.display.set_caption("Le Window")
# Loop until the user clicks the close button.
done = False
clock = pygame.time.Clock()
# -------- Main Program Loop -----------
while not done:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            done = True
    screen.fill(WHITE)
pygame.display.flip()
clock.tick(60) # Limit to 60 frames per second
# Close the window and quit.
pygame.quit()
